# Data Wrangling Notebook


## Summary of work to be done

The goal in this notebook is to prepare the lending club data for visualization and modeling. Data had 144 columns
with many of them with more than 50% missing values. Many of columns contained data leakage. Some of the categorical nominal columns had more than 15 unique values which had to be recategorized in order to prepare them for one-hot encoding.

### Steps in the notebook:
1. Setting things up:
   * Import data
   * Import dictionary and create preview dataframe
     

2. Removing data based on different criteria:
   * Remove columns that contain more than 50% missing values
   * Remove rows that contain 100% missing values
   * Remove columns that contain data leakage
   * Remove columns that contain a single unique value
   * Remove columns that contain redundant information
   * Remove columns that contain information that can be infered from other columns
     

3. Feature engineering:
   * Engineer datetime columns
   * Estimate missing values of highly correlated columns
   * Recategorize some nominal columns
   * Change dtypes of some object columns
   * Prepare the dependent variable

## Setting things up

In [1]:
import pandas as pd
import numpy as np

### Import data

*Read in the downloaded csv file (header row was in the second row of the csv file, and last two rows contained useless data so those three rows were skipped with the attributes in read_csv function):*

In [2]:
df = pd.read_csv("./LoanStats3a.csv", header=1, nrows=42536, low_memory=False)
df.shape

(42536, 144)

In [3]:
pd.set_option("display.max_rows", 144)
pd.set_option('max_colwidth', 5000)
df.tail().T

,42531,42532,42533,42534,42535
id,NaN,NaN,NaN,NaN,NaN
member_id,NaN,NaN,NaN,NaN,NaN
loan_amnt,3500,1000,2525,6500,5000
funded_amnt,3500,1000,2525,6500,5000
funded_amnt_inv,225,0,225,0,0
term,36 months,36 months,36 months,36 months,36 months
int_rate,10.28%,9.64%,9.33%,8.38%,7.75%
installment,113.39,32.11,80.69,204.84,156.11
grade,C,B,B,A,A
sub_grade,C1,B4,B3,A5,A3


### Import dictionary and create preview dataframe

*Read in the lending club dictionary with the descriptions of columns:*

In [4]:
dictionary = pd.read_excel("./LCDataDictionary.xlsx")
print(dictionary.columns.tolist())

['LoanStatNew', 'Description']


*Change the dictionary column names for convenience:*

In [5]:
dictionary.columns = ['name', 'description']
print(dictionary.columns.tolist())

['name', 'description']


*Create a dataframe with name, dtypes, first value, description as columns for an overview of descriptions that is more convenient than the dictionary dataframe:*

In [6]:
df_dtypes = pd.DataFrame(df.dtypes,columns=['dtypes'])
df_dtypes = df_dtypes.reset_index()
df_dtypes['name'] = df_dtypes['index']
df_dtypes = df_dtypes[['name','dtypes']]
df_dtypes['first value'] = df.loc[0].values
preview = df_dtypes.merge(dictionary, on='name',how='left')
preview.head()

,name,dtypes,first value,description
0,id,object,NaN,A unique LC assigned ID for the loan listing.
1,member_id,float64,NaN,A unique LC assigned Id for the borrower member.
2,loan_amnt,float64,5000,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
3,funded_amnt,float64,5000,The total amount committed to that loan at that point in time.
4,funded_amnt_inv,float64,4975,The total amount committed by investors for that loan at that point in time.


## Removing columns based on different criteria

*Show columns that have at leat one missing value:*

In [7]:
df.isnull().sum()[df.isnull().sum() > 0]

id                                            42535
member_id                                     42536
loan_amnt                                         1
funded_amnt                                       1
funded_amnt_inv                                   1
term                                              1
int_rate                                          1
installment                                       1
grade                                             1
sub_grade                                         1
emp_title                                      2627
emp_length                                     1113
home_ownership                                    1
annual_inc                                        5
verification_status                               1
issue_d                                           1
loan_status                                       1
pymnt_plan                                        1
url                                           42536
desc        

*Locate the source of bunch of ones:*

In [8]:
df.iloc[39784:39787,0:5]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv
39784,NaN,NaN,5000.0,5000.0,650.0
39785,NaN,NaN,7500.0,7500.0,800.0
39786,Loans that do not meet the credit policy,NaN,NaN,NaN,NaN


### Remove columns that contain more than 50% missing values

*Remove all columns that have more than 50% missing values:*

In [9]:
df = df.drop((df.isnull().sum()[df.isnull().sum() > (df.shape[0] / 2)]).index, axis=1)
df.shape

(42536, 53)

### Remove rows that contain 100% missing values

*Remove the single row with all missing values:*

In [10]:
df = df.dropna(how='all')
df.reset_index(inplace=True, drop=True)
df.shape

(42535, 53)

   ### Remove columns that contain data leakage, a single unique value, redundant information and information               that can be infered from other columns

*Inspect the descriptions for columns that:* 
* *leak information*
* *contain redundant information*
* *contain information that can be infered from other columns*
* *contain only one/mostly one unique value:*

In [11]:
preview.head(54)

,name,dtypes,first value,description
0,id,object,NaN,A unique LC assigned ID for the loan listing.
1,member_id,float64,NaN,A unique LC assigned Id for the borrower member.
2,loan_amnt,float64,5000,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
3,funded_amnt,float64,5000,The total amount committed to that loan at that point in time.
4,funded_amnt_inv,float64,4975,The total amount committed by investors for that loan at that point in time.
5,term,object,36 months,The number of payments on the loan. Values are in months and can be either 36 or 60.
6,int_rate,object,10.65%,Interest Rate on the loan
7,installment,float64,162.87,The monthly payment owed by the borrower if the loan originates.
8,grade,object,B,LC assigned loan grade
9,sub_grade,object,B2,LC assigned loan subgrade


*Drop the problematic columns:*

In [12]:
cols_rm = ["funded_amnt", "funded_amnt_inv", "sub_grade", "pymnt_plan",
           "desc", "zip_code", "out_prncp", "out_prncp_inv", "initial_list_status",
           "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "title",
           "total_rec_int", "total_rec_late_fee", "recoveries", "application_type", 
           "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt",
           "last_credit_pull_d", "hardship_flag", "debt_settlement_flag", "delinq_amnt", 
           "chargeoff_within_12_mths", "collections_12_mths_ex_med", "tax_liens",
           "acc_now_delinq", "policy_code", "emp_title"
          ]
df = df.drop(cols_rm, axis=1)
df.shape

(42535, 23)

## Feature engineering

### Engineer datetime columns

*Use the issue_d and earliest_cr_line columns to create a new column that will contain the age of the credit line in months, then remove the two original columns:*

In [13]:
columns = ["issue_d", "earliest_cr_line"]
df["issue_d"] = pd.to_datetime(df["issue_d"])
df["earliest_cr_line"] = pd.to_datetime(df["earliest_cr_line"])
df["age_cr_line"] = df["issue_d"] - df["earliest_cr_line"]
df.drop(columns, axis=1, inplace=True)
df["age_cr_line"] = df["age_cr_line"].dt.days
# at this point age_cr_line column contains null values which have to be removed:
rows_rm = df[df.age_cr_line.isnull()].index.to_list()
df.drop(rows_rm, inplace=True)
df.reset_index(inplace=True, drop=True)
df["age_cr_line"] = df.age_cr_line.apply(lambda x: int(round((x / 30.44), 0)))
df.shape

(42506, 22)

In [14]:
df.isnull().sum()[df.isnull().sum() > 0]

emp_length              1112
revol_util                61
pub_rec_bankruptcies    1336
dtype: int64

### Estimate missing values of highly correlated columns

*From the descriptions alone of the pub_rec and pub_rec_bankruptcies columns we can assume high correlation between the two columns since the latter is clearly a subset of the former:*

In [15]:
pub_der_recs = pd.DataFrame()
pub_der_recs["pub_rec"] = df.pub_rec
pub_der_recs["pub_rec_bankruptcies"] = df.pub_rec_bankruptcies
pub_der_recs.corr()

,pub_rec,pub_rec_bankruptcies
pub_rec,1.000000,0.841454
pub_rec_bankruptcies,0.841454,1.000000


*Hence, we can use pub_rec to fill in pub_rec_bankruptcies nulls with the following logic:*
* *where pub_rec = 0 => pub_rec_bankruptcies = 0*
* *pub_rec = 1 => pub_rec_bankruptcies = 1 (even though it can be 0 we use conservative estimate, same number)*
* *pub_rec = 2 => pub_rec_bankruptcies = 2 (even though it can be 0 or 1 we use conservative estimate, same number)*
* *pub_rec = 3 => pub_rec_bankruptcies = 2 (for pub_rec = 3 all the bankruptcy values that are not nill are 0)* 
* *pub_rec = 5 => remove the row since it's the only one*

In [16]:
df.drop(df[df.pub_rec == 5].index.to_list(), inplace=True)
df.reset_index(inplace=True, drop=True)

In [17]:
def f(x):
    if (x["pub_rec"] == 0) and np.isnan(x["pub_rec_bankruptcies"]):
        return 0
    elif(x["pub_rec"] == 1) and np.isnan(x["pub_rec_bankruptcies"]):
        return 1
    elif(x["pub_rec"] == 2) and np.isnan(x["pub_rec_bankruptcies"]):
        return 2
    elif(x["pub_rec"] == 3) and np.isnan(x["pub_rec_bankruptcies"]):
        return 2
    else:
        return x["pub_rec_bankruptcies"]

In [18]:
df["pub_rec_bankruptcies"] = df.apply(f, axis=1)

### Recategorize some nominal columns

*Four rows have NONE as home_ownership value so they will be removed as well as the rows that contain missing values for revol_util column:*

In [19]:
rows_rm = df[df.home_ownership == "NONE"].index.to_list() + df[df.revol_util.isnull()].index.to_list()
df.drop(rows_rm, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

(42440, 22)

*There are rows that contain "Does not meet the credit policy." in the loan_status column, let's remove those:*

In [20]:
rows_rm = df[df.loan_status == "Does not meet the credit policy. Status:Fully Paid"].index.to_list() + \
          df[df.loan_status == "Does not meet the credit policy. Status:Charged Off"].index.to_list()
df.drop(rows_rm, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

(39733, 22)

*All missing values in emp_length column will be replaced with "< 1 year" since a reasonable estimate is that people who retired or are unemployed have null as the emp_length value:*

In [21]:
def f(x):
    v = str(x)
    if v == "nan":
        return "< 1 year"
    else:
        return x

In [22]:
df["emp_length"] = df.emp_length.apply(f)
df.shape

(39733, 22)

*addr_state will be recategorized into four categories that represent regions in the US:*

In [23]:
west = ['AZ', 'CA', 'OR', 'UT', 'WA', 'CO', 'NV', 'AK', 'NM', 'NM', 'HI', 'WY', 'ID']
midwest = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN']
south = ['GA', 'NC', 'TX', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'MD', 'WV', 'DC', 'AR', 'OK', 'DE', 'MS', 'TN']
northeast = ['CT', 'NY', 'PA', 'NJ', 'RI', 'MA', 'VT', 'NH', 'ME']

In [24]:
def f(x):
    if x in west:
        return "west"
    elif x in midwest:
        return "midwest"
    elif x in south:
        return "south"
    else:
        return "northeast"

In [25]:
df["addr_state"] = df.addr_state.apply(f)

*verification_status column has 3 unique values Verified, Not Verified and Source Verified. Source Verified will
be change into Verified:*

In [26]:
def f(x):
    if x == "Source Verified":
        return "Verified"
    else:
        return x

In [27]:
df["verification_status"] = df.verification_status.apply(f)

### Change dtypes of some object columns

*int_rate and revol_util will have trailing % removed and casted to float64:*

In [28]:
def f(x):
    return x[:-1]

In [29]:
df["int_rate"] = df.int_rate.apply(f)
df["int_rate"] = df["int_rate"].astype(np.float64)
df["revol_util"] = df.revol_util.apply(f)
df["revol_util"] = df["revol_util"].astype(np.float64)

*term column will have preceding space and trailing word month removed and casted to int64:*

In [30]:
def f(x):
    return x.lstrip()[:-7]

In [31]:
df["term"] = df.term.apply(f)
df["term"] = df["term"].astype(np.int64)

### Prepare the dependent variable

*loan_status column will have the value "Fully Paid" changed to 1 and "Charged Off" to 0:*

In [32]:
def f(x):
    if x == "Fully Paid":
        return 1
    else:
        return 0

In [33]:
df["loan_status"] = df.loan_status.apply(f)